In [1]:
import torch
from torch import nn
from torch.nn import functional as F
net = nn.Sequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256,10))
X = torch.rand(2,20)
print(net(X))

tensor([[-0.2848, -0.0046,  0.1485,  0.0694,  0.1365, -0.2073, -0.0581, -0.2430,
         -0.1822,  0.0755],
        [-0.0355, -0.1446, -0.0055,  0.1890,  0.0432, -0.2958, -0.0015, -0.2517,
         -0.2625,  0.0646]], grad_fn=<AddmmBackward0>)


## 自定义MLP模块

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [8]:
net = MLP()
print(net(X))


tensor([[ 0.0525,  0.0458,  0.0582, -0.1313, -0.0395,  0.1608,  0.0548, -0.0069,
          0.2730, -0.0207],
        [ 0.0340, -0.0911,  0.0446, -0.1149,  0.0132, -0.0109,  0.0249, -0.0183,
          0.0216,  0.0984]], grad_fn=<AddmmBackward0>)


## 顺序块
_modules的主要优点是： 在模块的参数初始化过程中， 系统知道在_modules字典中查找需要初始化参数的子块

In [9]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [10]:
net = MySequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256,10))
net(X)

tensor([[-0.2156, -0.0827,  0.1650,  0.3288, -0.1081,  0.0334, -0.0309,  0.1844,
         -0.0979,  0.0877],
        [-0.1253, -0.0316,  0.1243,  0.2422, -0.1406, -0.0167,  0.0620,  0.1469,
         -0.0979,  0.2021]], grad_fn=<AddmmBackward0>)

## 在前向传播函数中执行代码
有时候，想使用常数参数，需要设置反向求梯度时，不求导

In [11]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()


In [13]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0516, grad_fn=<SumBackward0>)

## 混合搭配各种组合块方法

In [ ]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)
